## Preprocessing

In [228]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
import warnings
warnings.filterwarnings('ignore')

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

In [229]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'], axis = 1)
application_df.head()

In [230]:
# Determine the number of unique values in each column.
application_df.nunique()

In [231]:
# Dropping extra columns to see if the results change
# Drop the non-beneficial columns 'STATUS', 'ORGANIZATION', 'USE_CASE'
application_df = application_df.drop(columns = ['STATUS', 'ORGANIZATION', 'USE_CASE' ], axis = 1)
application_df.head()

In [232]:
# Determine the number of unique values in each column.
application_df.nunique()

In [233]:
# Look at APPLICATION_TYPE value counts for binning
application_vcs = application_df['APPLICATION_TYPE'].value_counts()
application_vcs

In [234]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_vcs[application_vcs < 528].index) 
cutoff_value = 528 

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

In [235]:
# Look at CLASSIFICATION value counts for binning
classification_vcs = application_df['CLASSIFICATION'].value_counts()
classification_vcs

In [236]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_vcs = classification_vcs[classification_vcs > 1]
classification_vcs

In [237]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_vcs[classification_vcs < 1500].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

In [238]:
# Convert categorical data to numeric with `pd.get_dummies`
numeric_data = pd.get_dummies(application_df)

In [239]:
# Split our preprocessed data into our features and target arrays
X = numeric_data.drop(['IS_SUCCESSFUL'], axis = 1)
y = numeric_data['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 58)

In [240]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [241]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 256
hidden_nodes_layer2 = 128
hidden_nodes_layer3 = 64
hidden_nodes_layer4 = 32


nn_model = tf.keras.models.Sequential()

# Hidden layer1
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1,
             input_dim=number_input_features, activation="relu")),tf.keras.layers.Dropout(0.5),  # Adding dropout for regularization

# Hidden layer2
nn_model.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer2, activation="relu")),
tf.keras.layers.Dropout(0.5),  # Adding dropout for regularization
    
# Hidden layer3
nn_model.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer3, activation="relu"))
tf.keras.layers.Dropout(0.5), # Adding dropout for regularization
    
# Hidden layer4
nn_model.add(tf.keras.layers.Dense(
    units=hidden_nodes_layer4, activation="relu"))
tf.keras.layers.Dropout(0.5),  # Adding dropout for regularization
    
# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn_model.summary()

In [242]:
# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)

In [243]:
# Compile the model
# nn_model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy']) choosing to use this adapted code below.
nn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001), loss="binary_crossentropy", metrics=["accuracy"])

In [244]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs = 100)

In [245]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose = 2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

In [246]:
# Export our model to HDF5 file
filepath = r'C:\Users\melbu\OneDrive\Documents\Class Activities\Assignments\Module 21\deep-learning-challenge/AlphabetSoupCharity.h5'
nn_model.save(filepath)

nn_model.save('AlphabetSoupCharity_Optimisation.h5')